In [10]:
from time import perf_counter
import numpy as np,os
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler, power_transform, MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score, RepeatedStratifiedKFold, GridSearchCV, cross_val_predict
from sklearn.utils import shuffle
from sklearn.metrics import matthews_corrcoef, precision_recall_curve, auc, accuracy_score
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler, EditedNearestNeighbours, TomekLinks
from imblearn.ensemble import RUSBoostClassifier, BalancedRandomForestClassifier, BalancedBaggingClassifier
from imblearn.pipeline import Pipeline
import seaborn as sns,matplotlib.pyplot as plt

from library.configs import IMBS, CLFS, ENSEMBLES, CV, SCORERS
from library.utils import evaluate, read_data

In [11]:
DATASETS = ['groovy-1_5_7.csv','jruby-1.4.0.csv','lucene-2.9.0.csv','jruby-1.7.0.preview1.csv','groovy-1_6_BETA_1.csv',
        'derby-10.2.1.6.csv','wicket-1.5.3.csv','camel-2.9.0.csv','camel-1.4.0.csv','activemq-5.8.0.csv']
DATASETS = [f for f in os.listdir("JIRA/") if 'csv' in f]
len(DATASETS)

32

In [12]:
models = {}
for im,samp in IMBS.items():
    for c,clf in CLFS.items():
        models[(im,c)] = Pipeline([('samp',samp),('clf',clf)])
    
models.keys(),len(models)

(dict_keys([('smote', 'dt'), ('smote', 'lr'), ('smote', 'nb'), ('smote', 'svm'), ('smote', 'knn'), ('smote', 'rf'), ('rus', 'dt'), ('rus', 'lr'), ('rus', 'nb'), ('rus', 'svm'), ('rus', 'knn'), ('rus', 'rf'), ('wilson', 'dt'), ('wilson', 'lr'), ('wilson', 'nb'), ('wilson', 'svm'), ('wilson', 'knn'), ('wilson', 'rf'), ('tomek', 'dt'), ('tomek', 'lr'), ('tomek', 'nb'), ('tomek', 'svm'), ('tomek', 'knn'), ('tomek', 'rf'), ('None', 'dt'), ('None', 'lr'), ('None', 'nb'), ('None', 'svm'), ('None', 'knn'), ('None', 'rf')]),
 30)

In [13]:
path = "Clean.csv"
cols = pd.MultiIndex.from_product([IMBS.keys(),CLFS.keys(),[f.__name__ for f in SCORERS]],names=['imb','clf','metric'])
df = pd.DataFrame(index=DATASETS,columns=cols)
#df = pd.read_csv(path,header=[0,1,2],index_col=0)

In [14]:
%%time
for it,d in enumerate(DATASETS):
    print(it)
    X,y_noisy,y_real = read_data(d,stats=True)
    for k in models:
        print(k)
        sd = perf_counter()
        r = evaluate(models[k],X,y_real,y_real,CV,SCORERS)
        for f in r:
            df.loc[d,(k[0],k[1],f)] = r[f].mean()
        print(round(perf_counter()-sd,2),[round(r[f].mean(),3) for f in r])
    print()
    df.to_csv(path)

0
activemq-5.8.0.csv noise:0.058, imb:15.847,203,3217, Shape:(3420, 65)
('smote', 'dt')
3.56 [0.19, 0.256]
('smote', 'lr')
10.38 [0.298, 0.317]
('smote', 'nb')
0.32 [0.29, 0.391]
('smote', 'svm')
165.59 [0.292, 0.234]
('smote', 'knn')
3.22 [0.251, 0.31]
('smote', 'rf')
15.1 [0.262, 0.306]
('rus', 'dt')
0.2 [0.212, 0.431]
('rus', 'lr')
1.25 [0.276, 0.312]
('rus', 'nb')
0.14 [0.288, 0.391]
('rus', 'svm')
1.18 [0.266, 0.253]
('rus', 'knn')
0.47 [0.241, 0.298]
('rus', 'rf')
1.85 [0.276, 0.278]
('wilson', 'dt')
7.16 [0.241, 0.336]
('wilson', 'lr')
9.7 [0.339, 0.356]
('wilson', 'nb')
6.61 [0.284, 0.422]
('wilson', 'svm')
19.25 [0.32, 0.346]
('wilson', 'knn')
8.1 [0.281, 0.295]
('wilson', 'rf')
9.95 [0.318, 0.352]
('tomek', 'dt')
6.04 [0.224, 0.292]
('tomek', 'lr')
8.8 [0.297, 0.355]
('tomek', 'nb')
5.17 [0.294, 0.395]
('tomek', 'svm')
23.49 [0.3, 0.334]
('tomek', 'knn')
6.91 [0.254, 0.285]
('tomek', 'rf')
9.85 [0.311, 0.368]
('None', 'dt')
1.12 [0.188, 0.261]
('None', 'lr')
3.56 [0.28, 0.354

/home/shihab/anaconda3/envs/ana/lib/python3.7/site-packages/sklearn/metrics/_classification.py:896: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


1.51 [0.26, 0.428]
('None', 'knn')
0.27 [0.459, 0.57]
('None', 'rf')
1.93 [0.518, 0.629]

2
activemq-5.3.0.csv noise:0.094, imb:15.669,142,2225, Shape:(2367, 65)
('smote', 'dt')
1.96 [0.278, 0.396]
('smote', 'lr')
5.46 [0.371, 0.47]
('smote', 'nb')
0.24 [0.337, 0.461]
('smote', 'svm')
74.48 [0.384, 0.434]
('smote', 'knn')
1.8 [0.28, 0.391]
('smote', 'rf')
8.84 [0.39, 0.481]
('rus', 'dt')
0.18 [0.264, 0.477]
('rus', 'lr')
1.17 [0.368, 0.465]
('rus', 'nb')
0.1 [0.335, 0.457]
('rus', 'svm')
1.57 [0.35, 0.441]
('rus', 'knn')
0.38 [0.306, 0.409]
('rus', 'rf')
1.82 [0.34, 0.453]
('wilson', 'dt')
3.84 [0.348, 0.484]
('wilson', 'lr')
5.38 [0.387, 0.485]
('wilson', 'nb')
3.58 [0.328, 0.477]
('wilson', 'svm')
10.26 [0.393, 0.469]
('wilson', 'knn')
4.24 [0.341, 0.42]
('wilson', 'rf')
6.34 [0.387, 0.479]
('tomek', 'dt')
3.71 [0.289, 0.399]
('tomek', 'lr')
5.7 [0.385, 0.49]
('tomek', 'nb')
3.14 [0.336, 0.455]
('tomek', 'svm')
15.85 [0.375, 0.474]
('tomek', 'knn')
4.27 [0.353, 0.431]
('tomek', 'rf')

/home/shihab/anaconda3/envs/ana/lib/python3.7/site-packages/sklearn/metrics/_classification.py:896: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/shihab/anaconda3/envs/ana/lib/python3.7/site-packages/sklearn/metrics/_classification.py:896: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/shihab/anaconda3/envs/ana/lib/python3.7/site-packages/sklearn/metrics/_classification.py:896: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/shihab/anaconda3/envs/ana/lib/python3.7/site-packages/sklearn/metrics/_classification.py:896: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


15.74 [0.096, 0.167]
('tomek', 'knn')


...

4.2 [0.272, 0.346]
('None', 'knn')
0.57 [0.24, 0.327]
('None', 'rf')
2.91 [0.254, 0.369]

18
hive-0.10.0.csv noise:0.126, imb:29.000,52,1508, Shape:(1560, 65)
('smote', 'dt')
1.62 [0.345, 0.458]
('smote', 'lr')
4.77 [0.453, 0.545]
('smote', 'nb')
0.21 [0.429, 0.503]
('smote', 'svm')
28.5 [0.399, 0.487]
('smote', 'knn')
1.02 [0.371, 0.514]
('smote', 'rf')
6.62 [0.522, 0.549]
('rus', 'dt')
0.15 [0.307, 0.508]
('rus', 'lr')
1.28 [0.401, 0.508]
('rus', 'nb')
0.1 [0.425, 0.498]
('rus', 'svm')
0.77 [0.413, 0.455]
('rus', 'knn')
0.23 [0.381, 0.488]
('rus', 'rf')
1.69 [0.384, 0.485]
('wilson', 'dt')
2.22 [0.393, 0.525]
('wilson', 'lr')
4.27 [0.477, 0.558]
('wilson', 'nb')
2.03 [0.41, 0.536]
('wilson', 'svm')
4.89 [0.454, 0.493]
('wilson', 'knn')
2.4 [0.444, 0.515]
('wilson', 'rf')
4.35 [0.476, 0.54]
('tomek', 'dt')
2.02 [0.361, 0.467]
('tomek', 'lr')
3.77 [0.453, 0.556]
('tomek', 'nb')
1.63 [0.433, 0.494]
('tomek', 'svm')
7.26 [0.419, 0.517]
('tomek', 'knn')
2.16 [0.42, 0.483]
('tomek', 'rf')


/home/shihab/anaconda3/envs/ana/lib/python3.7/site-packages/sklearn/metrics/_classification.py:896: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


103.97 [0.105, 0.21]
('tomek', 'knn')
46.37 [0.22, 0.211]
('tomek', 'rf')
43.5 [0.234, 0.281]
('None', 'dt')
2.52 [0.191, 0.217]
('None', 'lr')
7.83 [0.182, 0.246]
('None', 'nb')
0.27 [0.279, 0.352]
('None', 'svm')


/home/shihab/anaconda3/envs/ana/lib/python3.7/site-packages/sklearn/metrics/_classification.py:896: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/shihab/anaconda3/envs/ana/lib/python3.7/site-packages/sklearn/metrics/_classification.py:896: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/shihab/anaconda3/envs/ana/lib/python3.7/site-packages/sklearn/metrics/_classification.py:896: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/shihab/anaconda3/envs/ana/lib/python3.7/site-packages/sklearn/metrics/_classification.py:896: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/shihab/anaconda3/envs/ana/lib/python3.7/site-packages/sklearn/metrics/_classification.py:896: RuntimeWarning: invalid value encountered in double_scalars
  mcc = 

75.58 [0.053, 0.198]
('None', 'knn')
12.43 [0.214, 0.209]
('None', 'rf')
10.88 [0.23, 0.289]

20
derby-10.2.1.6.csv noise:0.290, imb:9.906,180,1783, Shape:(1963, 65)
('smote', 'dt')
1.92 [0.455, 0.697]
('smote', 'lr')
6.28 [0.57, 0.791]
('smote', 'nb')
0.71 [0.532, 0.751]
('smote', 'svm')
32.37 [0.583, 0.78]
('smote', 'knn')
1.9 [0.465, 0.727]
('smote', 'rf')
7.56 [0.602, 0.811]
('rus', 'dt')
0.61 [0.445, 0.7]
('rus', 'lr')
3.19 [0.575, 0.786]
('rus', 'nb')
0.13 [0.533, 0.751]
('rus', 'svm')
8.71 [0.57, 0.777]
('rus', 'knn')
0.8 [0.479, 0.731]
('rus', 'rf')
3.53 [0.582, 0.803]
('wilson', 'dt')
2.74 [0.455, 0.718]
('wilson', 'lr')
4.85 [0.541, 0.772]
('wilson', 'nb')
2.6 [0.543, 0.767]
('wilson', 'svm')
8.43 [0.541, 0.762]
('wilson', 'knn')
2.98 [0.471, 0.729]
('wilson', 'rf')
5.49 [0.554, 0.776]
('tomek', 'dt')
3.62 [0.465, 0.699]
('tomek', 'lr')
7.24 [0.581, 0.792]
('tomek', 'nb')
2.86 [0.54, 0.752]
('tomek', 'svm')
17.11 [0.573, 0.78]
('tomek', 'knn')
3.47 [0.489, 0.735]
('tomek', 'r

In [21]:
df.shape

(32, 42)

## Effect of Noise on Performance

In [ ]:
noise = pd.read_csv("Imb X Clf- Noise.csv",header=[0,1,2],index_col=0)
clean = pd.read_csv("Imb X Clf- Clean.csv",header=[0,1,2],index_col=0)
noise.shape,clean.shape

In [ ]:
wilcoxon(noise.values.reshape(-1),clean.values.reshape(-1))

In [ ]:
noise.values.mean(),clean.values.mean()

## Statistical Analysis

In [ ]:
from scipy.stats import wilcoxon, friedmanchisquare
import scikit_posthocs as sp

In [ ]:
boo = df.rename_axis('Datasets').reset_index()

In [ ]:
res = pd.melt(boo,id_vars=['Datasets'])
res.columns

In [ ]:
res

### Compare DT & RF & NB

In [ ]:
mathew =  res[res['metric']=='matthews_corrcoef'].drop(columns=['metric'])

In [ ]:
mathew = mathew[mathew['clf'].isin(['dt','rf','nb'])]
mathew.shape

In [ ]:
tmp = mathew.set_index(['Datasets','imb'])
tmp

In [ ]:
hey = tmp.pivot(columns='clf')
hey

In [ ]:
dog = hey.reset_index(drop=True)
dog.columns = dog.columns.droplevel(0)
dog

In [ ]:
friedmanchisquare(*(dog[c] for c in dog.columns))

In [ ]:
dog.mean()f

In [ ]:
pvals = pd.DataFrame(columns=dog.columns,index=dog.columns,dtype='float')
for c in dog.columns:
    for d in dog.columns:
        if c==d: continue
        pvals.loc[c,d] = float(wilcoxon(dog[c],dog[d]).pvalue)
pvals

In [ ]:
from statsmodels.stats.multitest import multipletests

In [ ]:
ps = pvals.values.reshape(-1)
idx = np.isnan(ps)
idx

In [ ]:
ps[~idx]

In [ ]:
multipletests(ps[~idx],method='fdr_by')